In [ ]:
from bs4 import BeautifulSoup
from datetime import date
import datetime
import requests
import re
import os

from time import sleep
today = date.today()
f1="%B %d, %Y"
f2="%I:%M %p | %B %d, %Y"

#Correcting the link
def correct_link(current_link):
  if current_link:
    if (current_link[0] == '/'): 
      link = 'https://nation.com.pk' + current_link
    elif current_link.startswith("https://nation.com.pk"): 
      link=current_link

    else:
      link='https://nation.com.pk'
  else:
    link = 'https://nation.com.pk'
  return str(link)


#Filtering of links
def filter_link(list_link):  
    list_set = set(list_link)  
    all_links = (list(list_set)) 
    return all_links


#File Creation
def create_file(title, article_paragraphs):

  title = re.sub('[^ a-zA-Z0-9:)]', '', title)  
  f_name = "".join(i+" " for i in re.findall("[A-Za-z0-9]+",title))
  f_name = f_name + "_" + ".txt"
  with open(f_name, "w", encoding="utf-8") as file_object:
    file_object.write(article_paragraphs)
  return
#Extracting All the links to the  news articles
def date_check(time) :
    try:
        datetime.datetime.strptime(time, f1)
        date=datetime.datetime.strptime(time, f1).date()
        #print(date)
    except :
        date=datetime.datetime.strptime(time,f2).date()
        #print(date)
    return date 
def all_links(url):
  source=requests.get(url)
  sleep(2)
  soup=BeautifulSoup(source.text,'html.parser')
  l=[]
  links=soup.find_all('a')
  for n in range(0,len(links)):
    sleep(2)
    partial_link=links[n].get('href')
    link = correct_link(partial_link)
    data = requests.get(link)
    soup = BeautifulSoup(data.text,'html.parser')
    sublinks = soup.find_all("a")
    #print(sublinks)
    if sublinks:
      for i in sublinks:
        try:
          href = i['href']
          s_link = correct_link(href)
          #print(link)
          l.append(s_link)
        except:
          continue
    l.append(link)
 
  
  return l
def parser(links_list):
  num=1
  print("Total Unique Links:",len(links_list))
  for i in range(0,len(links_list)):
    current_url=links_list[i] 
    
    try:
      
      source=requests.get(current_url)
      #sleep(2)
    except requests.exceptions.ConnectionError:
        #source.status_code = "Connection refused"
        continue

    
    soup=BeautifulSoup(source.content,'lxml')
    headline=soup.find("h1")

    arthur=soup.find('a',class_="authorname_coulum")

    time=soup.find('p', class_= 'meta-date')
    
    
    p_tags = soup.find_all('p')
    #print(i)
    
    para=[]

    
    if headline == None or time== None or arthur== None or  p_tags== None  :
      pass
    else:   
      
      for d in range(2,len(p_tags)-11):
        d_=p_tags[d].text
        para.append(d_)
      text="\n".join(para)  
      Current_Date = today.strftime("%Y-%m-%d")
      
      title=soup.find('title')
      time=time.text
      
      #print(current_url)
      #print(time)
      #time=datetime.datetime.strptime(time, '%B %d, %Y').date()
  
      
      time=date_check(time)
      time=str(time)
      Content_="Source: The Nation; "+"Link: " + current_url +"; " + "DateOfExtraction:"+ Current_Date + "; " + "DateOfPublication:" + time +"; " +"Authors:"  + arthur.text + "; " + "Title:" + title.text +"\n" +"Article:" + text
      #print(C
      if Content_:
        print(num,"Content Found on:",links_list[i])
        
      
        #title=soup.find('title')

        title=str(num)+":)"+ title.text
      
        create_file(title,Content_)
        num=num+1
        
  print("DONE:") 

def main():
  print(":::Welcome to The Nation Parser:::") 
  A_links=all_links("https://nation.com.pk")
  print("Total links:",len(A_links))
  f_links=filter_link(A_links)
  #f_links=["https://nation.com.pk/24-Dec-2020/politicians-who-chose-path-of-corruption-facing-humiliation-pm","https://nation.com.pk/23-Dec-2020/indian-provocations-threat-to-regional-peace-says-coas","https://nation.com.pk/22-Dec-2020/pm-urges-opposition-to-raise-issues-in-parliament"]
  print("Filtered links:",len(f_links))
  
  parser(f_links)


if __name__ == "__main__":
    main()  


  

